<a href="https://colab.research.google.com/github/jonsol/small_projects/blob/master/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')

In [ ]:
# Data description
print(dataset.tail())
print(dataset.describe())
print(dataset.isna().sum())

# Summary of categorical columns
cat_cols = dataset.select_dtypes(include=['object', 'category'])

for col in cat_cols:
    print(f"\nCategory counts for '{col}':")
    print(dataset[col].value_counts())

In [ ]:
# Code dummies for categorical variables

dataset['sex'] = dataset['sex'].map({1: 'male', 2: 'Europe', 3: 'Japan'})

In [ ]:
# Split in training and testing
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_dataset.describe().transpose()

In [ ]:
# Split feature from label
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('expenses')
test_labels = test_features.pop('expenses')

# Drop non-numeric columns before normalization
train_features_numeric = train_features.drop(columns=['sex', 'smoker', 'region'])
test_features_numeric = test_features.drop(columns=['sex', 'smoker', 'region'])


# Normalization
train_dataset.describe().transpose()[['mean', 'std']]

# Create Layer
normalizer = tf.keras.layers.Normalization(axis=-1)

# Fit the state of the preprocessing layer to the data
normalizer.adapt(np.array(train_features_numeric))

#Calculate the mean and variance, and store them in the layer:
print(normalizer.mean.numpy())

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
